Source: https://github.com/szppaks/pccomp_oct

In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import copy
import glob
import tqdm
import open3d
import os
import time
import utils.registration as registration
import utils.fread as fread
import utils.FCGF as FCGF
import utils.pointcloud as pointcloud

In [2]:
def octree_compress(points, depth):
    xmin, xmax = points[:, 0].min(), points[:, 0].max()
    ymin, ymax = points[:, 1].min(), points[:, 1].max()
    zmin, zmax = points[:, 2].min(), points[:, 2].max()
    
    xaxis = np.linspace(xmin, xmax, 2**depth + 1)
    yaxis = np.linspace(ymin, ymax, 2**depth + 1)
    zaxis = np.linspace(zmin, zmax, 2**depth + 1)
    
    xpos = np.searchsorted(xaxis, points[:, 0], side='right') - 1
    ypos = np.searchsorted(yaxis, points[:, 1], side='right') - 1
    zpos = np.searchsorted(zaxis, points[:, 2], side='right') - 1
    
    coords = xpos * (2**(depth*2)) + ypos * (2**depth) + zpos
    
    return coords, np.asarray([depth, xmin, xmax, ymin, ymax, zmin, zmax])

def octree_decompress(coords, params):
    depth = params[0].astype(np.int32)
    xmin, xmax = params[1], params[2]
    ymin, ymax = params[3], params[4]
    zmin, zmax = params[5], params[6]
    
    xaxis = np.linspace(xmin, xmax, 2**depth + 1)
    yaxis = np.linspace(ymin, ymax, 2**depth + 1)
    zaxis = np.linspace(zmin, zmax, 2**depth + 1)
    
    xpos = coords // (2**(depth*2))
    ypos = (coords - xpos * (2**(depth*2))) // (2**depth)
    zpos = coords - xpos * (2**(depth*2)) - ypos * (2**depth)
    
    x = xaxis[xpos]
    y = yaxis[ypos]
    z = zaxis[zpos]
    
    return np.stack([x, y, z], axis=-1)


def make_pcd(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    return pcd


def view_registered_point_clouds(source, target, transformation):
    p1 = copy.deepcopy(source)
    p2 = copy.deepcopy(target)
    
    p1.paint_uniform_color([1, 0.706, 0])
    p2.paint_uniform_color([0, 0.651, 0.929])
    
    p1.transform(transformation)
    
    o3d.visualization.draw_geometries([p1, p2])


def create_features(features_data):
    features = o3d.registration.Feature()
    features.data = features_data.T
    return features

In [3]:
def find_neighbors(source, target, voxel_size=0.03):
    matches = []
    pcd_tree = o3d.geometry.KDTreeFlann(source)
    target_points = np.asarray(target.points)
    
    for i in range(len(target.points)):
        k, idx, _ = pcd_tree.search_radius_vector_3d(target_points[i], voxel_size)
        if k > 0:
            matches.append([i, idx[0]])
            
    return matches

 
def calc_overlap_ratio(source, target, transformation, voxel_size=0.03):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    
    source_temp.transform(transformation)
    matches = find_neighbors(source_temp, target_temp, voxel_size)
    return len(matches) * 100 / min(len(source.points), len(target.points))

In [14]:
def compress_and_decompress(pcd, depth):
    points = np.asarray(pcd.points)
    # measure the time for compression
    start_t = time.time()
    coords, parms = octree_compress(points, depth)
    compression_time = time.time() - start_t
    start_t = time.time()
    cpoints = octree_decompress(coords, parms)
    decompression_time = time.time() - start_t
    return pointcloud.make_pcd(cpoints)
    # return compression_time, decompression_time

## Compression on Global Registration Accuracy

### Preparing dataset

In [10]:
root_feature_dir = "data/features"

experiment = "exp_12"
trial = "trial_1"
subject = "subject-1"
sequence = "01"

voxel_size = 0.03
depth = 8


file_name = f"{experiment}__{trial}__{subject}__{sequence}"

In [16]:
groundtruth_data = np.load(os.path.join("data/trajectories/groundtruth", experiment, f"{file_name}.gtpose.npz"))
    
feature_dir = os.path.join(root_feature_dir, experiment, trial, str(voxel_size), subject, sequence)

sequence_ts = fread.get_timstamps(feature_dir, ext=".secondary.npz")
delta = (sequence_ts[1:] - sequence_ts[:-1])
target_fps = 1e3 / np.mean(delta)

print(f"Target FPS: {target_fps}")
num_frames = len(sequence_ts)

sequence_inds = np.arange(0, num_frames, int(target_fps * 0.8))
groundtruth_t = groundtruth_data["local_t"]


estimated_t = []
execution_times = []

for t in sequence_inds:
    source_feature_file = os.path.join(feature_dir, f"{sequence_ts[t]}.secondary.npz")
    target_feature_file = os.path.join(feature_dir, f"{sequence_ts[t]}.global.npz")
    
    source = FCGF.get_features(source_feature_file, voxel_size, pcd_only=True)
    target = FCGF.get_features(target_feature_file, voxel_size, pcd_only=True)
    
    if calc_overlap_ratio(source, target, groundtruth_t[t], voxel_size) < 30:
        print(f"Overlap ratio too low at frame {t}, ")
        continue
    
    try:
        registration.view(compress_and_decompress(source, 8), target, groundtruth_t[t])
    except IndexError:
        print(f"Index error at frame {t}")
        continue



# for sequence in sequences:
#     sequence_pcds = pickle.load(open(sequence, "rb"))
    
#     sequence_id = os.path.basename(sequence).split("_")[1].split(".")[0]
    
#     compressed_pcds = []

#     for i in tqdm.trange(len(sequence_pcds)):
#         try:
#             points = sequence_pcds[i]
#             points = np.asarray(preprocess(make_pcd(points)).points)
#             coords, parms = octree_compress(points, depth)
#             cpoints = octree_decompress(coords, parms)
#             compressed_pcds.append(cpoints)
#         except IndexError:
#             compressed_pcds.append(sequence_pcds[i])
#             continue

Target FPS: 27.743394937897477
Overlap ratio too low at frame 0, 


In [6]:
pointclouds_dir = "data/point_clouds"
experiment = "exp_12"

voxel_size = 0.05
depth = 8

In [17]:
for trial in os.listdir(os.path.join(pointclouds_dir, experiment)):
    for subject in os.listdir(os.path.join(pointclouds_dir, experiment, trial)):
        for sequence in os.listdir(os.path.join(pointclouds_dir, experiment, trial, subject)):
            print(f"Processing {experiment}__{trial}__{subject}__{sequence}")
            
            sequence_dir = os.path.join(pointclouds_dir, experiment, trial, subject, sequence)

            secondary_pcd_files = glob.glob(os.path.join(sequence_dir, "*secondary.pcd"))
            
            out_dir = sequence_dir.replace(experiment, f"{experiment}_compressed")

            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
                
            for secondary_pcd_file in tqdm.tqdm(secondary_pcd_files):
                pcd = open3d.io.read_point_cloud(secondary_pcd_file)
                pcd = open3d.voxel_down_sample(pcd, voxel_size)
                try:
                    pcd = compress_and_decompress(pcd, depth)
                    open3d.io.write_point_cloud(os.path.join(out_dir, os.path.basename(secondary_pcd_file)), pcd)
                except IndexError:
                    continue

Processing exp_12__trial_1__subject-1__01


100%|██████████| 651/651 [00:07<00:00, 88.49it/s] 


Processing exp_12__trial_1__subject-1__02


100%|██████████| 654/654 [00:07<00:00, 83.73it/s] 


Processing exp_12__trial_1__subject-1__03


...

100%|██████████| 651/651 [00:07<00:00, 88.86it/s] 


Processing exp_12__trial_3__subject-1__04


100%|██████████| 642/642 [00:07<00:00, 88.94it/s] 


Processing exp_12__trial_3__subject-1__05


100%|██████████| 649/649 [00:07<00:00, 84.17it/s] 


In [5]:
pcd_files = glob.glob("data/point_clouds/exp_12/trial_1/subject-1/01/*.secondary.pcd")

### Reconstruction quality

In [29]:
pcd_files = glob.glob("../../local-registration/data/point_clouds/exp_10/trial_1/subject-1/01/*.secondary.pcd")

In [10]:
compressed_t, decompressed_t = [], []

for i in range(len(pcd_files)):
    try:
        pcd = open3d.io.read_point_cloud(pcd_files[i])
        t1, t2 = compress_and_decompress(pcd, 8)
        
        compressed_t.append(t1)
        decompressed_t.append(t2)
    except IndexError:
        continue
    
print(f"Compression time: {np.mean(compressed_t)}")
print(f"Decompression time: {np.mean(decompressed_t)}")

Compression time: 0.0023226117132624786
Decompression time: 0.00030495720027224124


In [63]:
compression_ratios = []

for i in tqdm.trange(len(pcd_files)):
    pcd = o3d.io.read_point_cloud(pcd_files[i])
    points = np.asarray(pcd.points)

    original_bytes = points.nbytes

    pcd = o3d.geometry.voxel_down_sample(pcd, voxel_size=0.05)

    points = np.asarray(pcd.points)

    compressed_bytes = points.nbytes

    compression_ratios.append(original_bytes / compressed_bytes)

100%|██████████| 369/369 [00:09<00:00, 39.34it/s]


In [64]:
print(f"Min compression ratio: {np.min(compression_ratios):.2f}")
print(f"Median compression ratio: {np.median(compression_ratios):.2f}")
print(f"Max compression ratio: {np.max(compression_ratios):.2f}")

Min compression ratio: 17.33
Median compression ratio: 36.52
Max compression ratio: 110.13


In [73]:
compression_ratios = []

for i in tqdm.trange(len(pcd_files)):
    pcd = o3d.io.read_point_cloud(pcd_files[i])
    points = np.asarray(pcd.points)
    original_bytes = points.nbytes
    
    pcd = o3d.geometry.voxel_down_sample(pcd, voxel_size=0.03)
    points = np.asarray(pcd.points)

    try: 
        coords, params = octree_compress(points, depth=6)
        coords = np.unique(coords)
        comp_points = octree_decompress(coords, params)
        
        compressed_bytes = comp_points.nbytes
        compression_ratios.append(original_bytes / compressed_bytes)
    except IndexError:
        continue
    

100%|██████████| 369/369 [00:10<00:00, 34.90it/s]


In [74]:
print(f"Min compression ratio: {np.min(compression_ratios):.2f}")
print(f"Median compression ratio: {np.median(compression_ratios):.2f}")
print(f"Max compression ratio: {np.max(compression_ratios):.2f}")


Min compression ratio: 32.77
Median compression ratio: 57.30
Max compression ratio: 109.83


In [61]:
rmse = []


for i in tqdm.trange(len(pcd_files)):
    pcd = o3d.io.read_point_cloud(pcd_files[i])
    # pcd = o3d.geometry.voxel_down_sample(pcd, voxel_size=0.03)
    points = np.asarray(pcd.points)
    compressed_bytes = points.nbytes

    try: 
        coords, params = octree_compress(points, depth=10)
        comp_points = octree_decompress(coords, params)
        rmse.append(np.sqrt(np.mean((points - comp_points)**2)))
    except IndexError:
        continue
    

100%|██████████| 369/369 [00:16<00:00, 21.89it/s]


In [62]:
print(f"Mean RMSE: {np.max(rmse):.3f}")

Mean RMSE: 0.030
